In [1]:
# Run While Mounting a Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.6 MB/s eta 0:00:00


In [6]:
#boundry box updated code

import cv2
import numpy as np
from ultralytics import YOLO

# --- 1. SETUP ---
# Load the pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')

# --- 2. EDIT THESE ---
# Paste the exact path to YOUR video file
input_video_path = '/content/drive/MyDrive/AI_based_Intellegent_Traffic_Monitoring_and_Management_System/Data/input.mp4'

# Define the coordinates of YOUR lane's polygon
ROI_POINTS = np.array([(150, 100), (1300, 150), (1300, 600), (150, 600)], np.int32)
# --- END OF EDIT SECTION ---

# Define the output path
output_video_path = 'output_vehicle_count.mp4'

# COCO class IDs for vehicles (car, motorcycle, bus, truck)
VEHICLE_CLASS_IDS = [2, 3, 5, 7]

# --- 3. VIDEO PROCESSING ---
cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

print("Processing video... This may take a moment.")

try:
    while True:
        success, frame = cap.read()
        if not success:
            break # End of video

        # This check prevents errors if a frame is corrupted
        if frame.shape[1] != frame_width or frame.shape[0] != frame_height:
            frame = cv2.resize(frame, (frame_width, frame_height))

        # Reset counter for each frame
        vehicle_count = 0

        # Draw the ROI polygon once per frame
        cv2.polylines(frame, [ROI_POINTS], isClosed=True, color=(255, 0, 0), thickness=2)

        # Run YOLO detection
        results = model(frame)


        for box in results[0].boxes:
          cls_id = int(box.cls)
          if cls_id in VEHICLE_CLASS_IDS:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            if cv2.pointPolygonTest(ROI_POINTS, (center_x, center_y), False) >= 0:
              vehicle_count += 1
              label = model.names[cls_id]  # e.g., 'car', 'bus'

              # Draw bounding box
              cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

              # Put label above the bounding box
              cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                          0.8, (0, 255, 0), 2)


        # Write the final count and the frame to the output video
        cv2.putText(frame, f"Vehicles in Lane: {vehicle_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        out.write(frame)

finally:
    # This ensures files are closed properly even if an error occurs
    print("Processing complete. Releasing resources.")
    cap.release()
    out.release()

Processing video... This may take a moment.

0: 384x640 4 persons, 9 cars, 5 motorcycles, 3 trucks, 159.8ms
Speed: 3.8ms preprocess, 159.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 9 cars, 5 motorcycles, 3 trucks, 137.5ms
Speed: 2.8ms preprocess, 137.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 9 cars, 5 motorcycles, 3 trucks, 170.9ms
Speed: 2.6ms preprocess, 170.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 9 cars, 5 motorcycles, 3 trucks, 139.9ms
Speed: 2.7ms preprocess, 139.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 9 cars, 5 motorcycles, 3 trucks, 156.3ms
Speed: 2.9ms preprocess, 156.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 9 cars, 5 motorcycles, 3 trucks, 148.4ms
Speed: 3.0ms preprocess, 148.4ms inference, 1.6ms postprocess per image at shape (